# Extracting paths from temporal networks

[Run notebook in Google Colab](https://colab.research.google.com/github/pathpy/pathpy/blob/master/doc/tutorial/path_extraction.ipynb)  
[Download notebook](https://github.com/pathpy/pathpy/raw/master/doc/tutorial/path_extraction.ipynb)

This short tutorial demonstrates (and explains) how to calculate time-respecting path frequencies in a temporal network.

pip install git+git://github.com/pathpy/pathpy.git

In [1]:
import pathpy as pp
import io
import numpy as np

We first generate a maximally simple temporal network with three (instantaneous) time-stamped edges:

In [3]:
tn = pp.TemporalNetwork()
tn.add_edge('a', 'b', timestamp=1)
tn.add_edge('b', 'c', timestamp=2)
tn.add_edge('b', 'd', timestamp=5)
tn.plot()

As a first step, we can turn this temporal network into a time-unfolded directed acyclic graph. For this, we have to specify the maximum time difference delta between any two time-stamped edges that shall constitute a time-respecting or causal path. In addition to occuring within the maximum time difference, time-stamped edges also have to occur in the correct temporal ordering.

In the resulting time-unfolded directed acyclic graph, each time-unfolded node `v_t` represents a node `v` at a given time stamp `t`. Each edge (`v_t`, `w_{t'}`) between such time-unfolded nodes represents a possible causal influence (i.e. a time-respecting of causal path) by which node `v` at time `t` can influence node `w` at `t'>t`.

By definition, each time-stamped edge (`v`, `w`, t) is a causal path of length one by which node `v` at time `t` can influence node `w` at the next timestamp `t+1` (i.e. we assume that it takes one unit of time for influence to traverse an edge). For a maximum time difference of one between two edges, the only causal path of length two connects node `a` (at time 1) via node `b` (at time 2) to node `c` at time 3. We can see this in the resulting time-unfolded directed acyclic graph:

In [4]:
dag = pp.DirectedAcyclicGraph.from_temporal_network(tn, delta=1)
dag.plot()

If we increase the maximum time difference to `delta=2` three additional time-respecting paths of length one emerges (one from `b` at time 5 to `d` at time 7,  one from `a` at time 1 to `b` at time 3, and one from node `b` at time 2 to node `c` at time 4). This further implies one additional time-respecting path of length two, which is represented in the DAG below:

In [5]:
dag = pp.DirectedAcyclicGraph.from_temporal_network(tn, delta=2)
dag.plot()

If we set the delta to a maximum value of `infinity`, only the time-ordering of time-stamped edges is considered, i.e. any time gap between edges is allowed. In the example above, this implies that the state of node `a` at time `t=1` can influence any other node at any later time. In the directed acyclic graph this is represented as:

In [6]:
dag = pp.DirectedAcyclicGraph.from_temporal_network(tn, delta=np.inf)
dag.plot()

Thanks to its acyclicity, a directed acyclic graph can be used to calculate a finite set of all paths from any root node (the potential start node/time of a causal path in a temporal network) to any leaf node (the potential end node/time of a causal path) in the DAG. We can use the `roots` and `leafs` properties of the `DirectedAcyclicGraph` class to return those:

In [7]:
print([v.uid for v in dag.roots])
print([v.uid for v in dag.leafs])

['a_1']
['c_6', 'b_6', 'c_4', 'b_4', 'd_6', 'c_5', 'c_3', 'b_3']


We can calculate all possible paths from a given root node to any leaf node i nthe DAG as follows:

In [8]:
paths = dag.routes_from('a_1')
paths

Counter({('a_1', 'b_6'): 1,
         ('a_1', 'b_4'): 1,
         ('a_1', 'b_3'): 1,
         ('a_1', 'b_2', 'c_6'): 1,
         ('a_1', 'b_2', 'c_4'): 1,
         ('a_1', 'b_2', 'c_5'): 1,
         ('a_1', 'b_2', 'c_3'): 1,
         ('a_1', 'b_5', 'd_6'): 1})

To calculate the statistics of all paths from all roots, we can call the following function. In the example above, this is identical to the paths from the root `a_1` since there is a single root node only in this DAG.

In [9]:
paths = pp.algorithms.path_extraction.all_paths_from_dag(dag)
paths

Counter({('a_1', 'b_6'): 1,
         ('a_1', 'b_4'): 1,
         ('a_1', 'b_3'): 1,
         ('a_1', 'b_2', 'c_6'): 1,
         ('a_1', 'b_2', 'c_4'): 1,
         ('a_1', 'b_2', 'c_5'): 1,
         ('a_1', 'b_2', 'c_3'): 1,
         ('a_1', 'b_5', 'd_6'): 1})

The problem with this is that we are actually not interested in paths in the time-unfolded directed acyclic graph, but in causal, i.e. time-respecting paths in the original network. In our example, different nodes in the directed acyclic graph actually correspond to the same node in the network at different times. For instance, nodes `b_4` and `b_6` represent the same node `b` at time `4` and `6`. 

For the calculation of paths in the original network, we must incorporate this information. pathpy supports this via a custom node mapping, that we can pass to the path calculation. Moreover, each node in the directed acyclic graph generated by the `from_temporal_network` function has a node attribute `original` that contains the ID of the original node in the temporal network. To map DAG nodes to such nodes we can thus write:

In [10]:
paths = pp.algorithms.path_extraction.all_paths_from_dag(dag, node_mapping={ v.uid: v['original'].uid for v in dag.nodes })
paths

Counter({('a', 'b'): 1, ('a', 'b', 'c'): 1, ('a', 'b', 'd'): 1})

We have now mapped four different paths `a->b->c` to a single causal path, because the different paths in the original DAG all represent the same path from a single root node to the same (mapped) leaf node. The only difference is that transitions between nodes happen at different times.

It might appear that this is all we need to calculate statistics of causal paths in a temporal network. However, the situation is more complicated if we additionally consider which of the shorter paths are already contained in the longer paths. Let us reconsider the DAG generated above:

In [11]:
dag.plot()

In the original temporal network, there are actually only three temporal edges `(a,b), (b,c)` and `(c,d)` occurring in sequence. This leads to two different longest causal paths of length two, which contain the three shorter causal paths of length one (i.e. the edges).

The idea to focus on longest causal paths only is the basis for the calculation of poath statistics using the following function:

In [12]:
paths = pp.algorithms.path_extraction.all_paths_from_temporal_network(tn, delta=np.inf)
paths

Counter({('a', 'b', 'c'): 1, ('a', 'b', 'd'): 1})

In the example above, we only have a single root node, which is why the path statistics for the whole DAG is identical to the statistics returned for the paths originating in the single root note. 

In our temporal network with delta=2 we actually have two different roots, hence we have to consider causal paths starting in different nodes:

In [13]:
dag = pp.DirectedAcyclicGraph.from_temporal_network(tn, delta=2)
dag.plot()

In [14]:
paths = dag.routes_from('a_1')
paths

Counter({('a_1', 'b_3'): 1,
         ('a_1', 'b_2', 'c_3'): 1,
         ('a_1', 'b_2', 'c_4'): 1})

In [15]:
paths = dag.routes_from('b_5')
paths

Counter({('b_5', 'd_7'): 1, ('b_5', 'd_6'): 1})

In [16]:
paths = pp.algorithms.path_extraction.all_paths_from_dag(dag)
paths

Counter({('b_5', 'd_7'): 1,
         ('b_5', 'd_6'): 1,
         ('a_1', 'b_3'): 1,
         ('a_1', 'b_2', 'c_3'): 1,
         ('a_1', 'b_2', 'c_4'): 1})

In [17]:
paths = pp.algorithms.path_extraction.all_paths_from_temporal_network(tn, delta=2)
paths

Counter({('a', 'b', 'c'): 1, ('b', 'd'): 1})

In [18]:
tn = pp.TemporalNetwork()
tn.add_edge('a', 'b', timestamp=1)
tn.add_edge('b', 'c', timestamp=2)
tn.add_edge('b', 'd', timestamp=5)
tn.add_edge('a', 'b', timestamp=12)
tn.add_edge('b', 'c', timestamp=13)
paths = pp.algorithms.path_extraction.all_paths_from_temporal_network(tn, delta=2)
paths

Counter({('a', 'b', 'c'): 2, ('b', 'd'): 1})

In [19]:
pp.io.infomap.to_state_file(paths, 'test.state', weight=True)

In [20]:
with io.open('test.state', 'r') as f:
    print(f.read())

*Vertices 4
1 "d"
2 "a"
3 "c"
4 "b"
*States
1 2 "{eps}_a"
2 4 "{a}_b"
3 3 "{a-b}_c"
4 4 "{eps}_b"
5 1 "{b}_d"
*Links
1 2 2
2 3 2
4 5 1


In [21]:
n = pp.io.konect.read_konect_name('sociopatterns-hypertext')
print(n)

Uid:			0x2a1800677b8
Type:			TemporalNetwork
Directed:		False
Multi-Edges:		True
Number of unique nodes:	113
Number of unique edges:	2196
Number of temp nodes:	113
Number of temp edges:	20818
Observation periode:	1246255220 - 1246467561.0

Network attributes
------------------
category:	HumanContact
code:	HY
name:	Hypertext 2009
description:	Visitorâ€“visitor face-to-face contacts
extr:	sociopatterns
url:	http://www.sociopatterns.org/
long-description:	This is the network of face-to-face contacts of the attendees of the ACM Hypertext 2009 conference. The ACM Conference on Hypertext and Hypermedia 2009 (HT 2009, http://www.ht2009.org/) was held in Turin, Italy over three days from June 29 to July 1, 2009. In the network, a node represents a conference visitor, and an edge represents a face-to-face contact that was active for at least 20 seconds. Multiple edges denote multiple contacts. Each edge is annotated with the time at which the contact took place.
entity-names:	visitor
relationsh

In [22]:
validation, train = pp.algorithms.evaluation.train_test_split(n, train_size = 0.8, split = "time")

In [22]:
train_paths = pp.algorithms.path_extraction.all_paths_from_temporal_network(train, delta=1)
train_paths

KeyboardInterrupt: 

# PaCO: Path Counting in Temporal Networks

We will look at two temporal netorks:

In [23]:
tn1 = pp.TemporalNetwork(directed=True)
tn1.add_edge("a", "b", timestamp=1)  # 0
tn1.add_edge("a", "b", timestamp=2)  # 1
tn1.add_edge("b", "a", timestamp=3)  # 2
tn1.add_edge("b", "c", timestamp=3)  # 3
tn1.add_edge("d", "c", timestamp=3)  # 4
tn1.add_edge("d", "c", timestamp=4)  # 5
tn1.add_edge("c", "d", timestamp=5)  # 6
tn1.add_edge("c", "b", timestamp=6)  # 7
tn1.add_edge("b", "c", timestamp=7)  # 8
tn1.plot()

In [24]:
tn2 = pp.TemporalNetwork(directed=True)
tn2.add_edge("a", "b", timestamp=1)  # 0
tn2.add_edge("a", "c", timestamp=2)  # 1
tn2.add_edge("b", "c", timestamp=2)  # 2
tn2.add_edge("c", "d", timestamp=3)  # 3
tn2.add_edge("b", "d", timestamp=4)  # 4
tn2.add_edge("d", "c", timestamp=4)  # 5
tn2.add_edge("d", "c", timestamp=5)  # 6
tn2.add_edge("d", "a", timestamp=5)  # 7
tn2.add_edge("c", "b", timestamp=6)  # 8
tn2.plot()

For PaCo, we define paths in temporal networks to consist of temporal links which 
* can be continued topologically: the destination of a previous link is the same as the source of the next link, e.g. $(a,b,t_1)$ and $(b, e, t_2)$.
* can be continued temporally: the timestamps of successive links $t_i$, $t_{i+1}$ satisfy
$$t_i < t_{i+1}$$
$$t_{i+1} - t_i \leq \delta $$

For the first temporal network, and choices $\delta = 2$ and $\delta = 3$ the paths are as follows:

In [25]:
tn1_delta2= {
    1: {('a', 'b'): 2,
        ('b', 'a'): 1,
        ('b', 'c'): 2,
        ('c', 'b'): 1,
        ('c', 'd'): 1,
        ('d', 'c'): 2},
    2: {('a', 'b', 'a'): 2,
        ('a', 'b', 'c'): 2,
        ('b', 'c', 'd'): 1,
        ('c', 'b', 'c'): 1,
        ('d', 'c', 'b'): 1,
        ('d', 'c', 'd'): 2},
    3: {('a', 'b', 'c', 'd'): 2,
        ('d', 'c', 'b', 'c'): 1}
}

In [26]:
tn1_delta3 =  {
    1: {('a', 'b'): 2,
        ('b', 'a'): 1,
        ('b', 'c'): 2,
        ('c', 'b'): 1,
        ('c', 'd'): 1,
        ('d', 'c'): 2},
    2: {('a', 'b', 'a'): 2,
        ('a', 'b', 'c'): 2,
        ('b', 'c', 'b'): 1,
        ('b', 'c', 'd'): 1,
        ('c', 'b', 'c'): 1,
        ('d', 'c', 'b'): 2,
        ('d', 'c', 'd'): 2},
    3: {('a', 'b', 'c', 'b'): 2,
        ('a', 'b', 'c', 'd'): 2,
        ('b', 'c', 'b', 'c'): 1,
        ('d', 'c', 'b', 'c'): 2},
    4: {('a', 'b', 'c', 'b', 'c'): 2}
}

For the second temporal network and choices $\delta = 1$ and $\delta = 2$  the paths are following:

In [27]:

tn2_delta1 = {
    1: {('a', 'b'): 1,
        ('a', 'c'): 1,
        ('b', 'c'): 1,
        ('b', 'd'): 1,
        ('c', 'b'): 1,
        ('c', 'd'): 1,
        ('d', 'a'): 1,
        ('d', 'c'): 2},
    2: {('a', 'b', 'c'): 1,
        ('a', 'c', 'd'): 1,
        ('b', 'c', 'd'): 1,
        ('b', 'd', 'a'): 1,
        ('b', 'd', 'c'): 1,
        ('c', 'd', 'c'): 1,
        ('d', 'c', 'b'): 1},
    3: {('a', 'b', 'c', 'd'): 1,
        ('a', 'c', 'd', 'c'): 1,
        ('b', 'c', 'd', 'c'): 1,
        ('b', 'd', 'c', 'b'): 1},
    4: {('a', 'b', 'c', 'd', 'c'): 1}
}

In [28]:
tn2_delta2 = {
    1: {('a', 'b'): 1,
        ('a', 'c'): 1,
        ('b', 'c'): 1,
        ('b', 'd'): 1,
        ('c', 'b'): 1,
        ('c', 'd'): 1,
        ('d', 'a'): 1,
        ('d', 'c'): 2},
    2: {('a', 'b', 'c'): 1,
        ('a', 'c', 'd'): 1,
        ('b', 'c', 'd'): 1,
        ('c', 'd', 'c'): 2,
        ('b', 'd', 'c'): 1,
        ('c', 'd', 'a'): 1,
        ('b', 'd', 'a'): 1,
        ('d', 'c', 'b'): 2},
    3: {('a', 'b', 'c', 'd'): 1,
        ('a', 'c', 'd', 'c'): 2,
        ('b', 'c', 'd', 'c'): 2,
        ('b', 'd', 'c', 'b'): 1,
        ('a', 'c', 'd', 'a'): 1,
        ('b', 'c', 'd', 'a'): 1,
        ('c', 'd', 'c', 'b'): 2},
    4: {('a', 'b', 'c', 'd', 'a'): 1,
        ('a', 'c', 'd', 'c', 'b'): 2,
        ('b', 'c', 'd', 'c', 'b'): 2,
        ('a', 'b', 'c', 'd', 'c'): 2},
    5: {('a', 'b', 'c', 'd', 'c', 'b'): 2}
}

PaCo finds these paths, and outputs them in a `PathCollection`.

In [31]:
def test_PaCo():
    """
    Test the PaCo algorithm
    """
    for tn, delta, solution in [ (tn1, 3, tn1_delta3), (tn1, 3, tn1_delta3), (tn2, 1, tn2_delta1), (tn2, 2, tn2_delta2)]: 
        PaCo_paths = pp.algorithms.path_extraction.PaCo(tn, delta, skip_first=0, up_to_k=10)
        for l in solution:
            for path in solution[l]:
                assert PaCo_paths[path]["frequency"] == solution[l][path], f"Mismatch in counts for path {path}, correct counter is {solution[l][path]}, PaCo computed {PaCo_paths[path]['frequency']}."

                PaCo_paths.remove(path)
        assert len(PaCo_paths) == 0, f"PaCo found some non-existing paths."
    return True

In [32]:
test_PaCo()

True

In [50]:
paths = pp.algorithms.path_extraction.PaCo(train, 300, skip_first=0, up_to_k=10)

In [51]:
from collections import Counter
cnt = Counter()
for path in paths:
    p = tuple([v.uid for v in path.nodes])
    cnt[p] = paths[path]['frequency']

In [56]:
pp.io.infomap.to_state_file(cnt, 'test.state')

In [57]:
with io.open('test.state', 'r') as f:
    print(f.read())

*Vertices 111
1 "68"
2 "58"
3 "100"
4 "53"
5 "108"
6 "110"
7 "101"
8 "11"
9 "73"
10 "4"
11 "70"
12 "10"
13 "6"
14 "74"
15 "32"
16 "12"
17 "75"
18 "105"
19 "1"
20 "77"
21 "3"
22 "2"
23 "27"
24 "59"
25 "93"
26 "98"
27 "87"
28 "66"
29 "9"
30 "46"
31 "21"
32 "106"
33 "26"
34 "89"
35 "88"
36 "97"
37 "90"
38 "95"
39 "76"
40 "39"
41 "84"
42 "38"
43 "67"
44 "55"
45 "20"
46 "16"
47 "102"
48 "13"
49 "8"
50 "49"
51 "65"
52 "35"
53 "94"
54 "48"
55 "36"
56 "45"
57 "23"
58 "28"
59 "25"
60 "103"
61 "83"
62 "42"
63 "78"
64 "92"
65 "50"
66 "5"
67 "96"
68 "64"
69 "52"
70 "71"
71 "19"
72 "62"
73 "72"
74 "29"
75 "41"
76 "56"
77 "33"
78 "79"
79 "17"
80 "43"
81 "14"
82 "37"
83 "91"
84 "81"
85 "104"
86 "85"
87 "30"
88 "15"
89 "86"
90 "7"
91 "24"
92 "47"
93 "80"
94 "22"
95 "18"
96 "57"
97 "107"
98 "109"
99 "34"
100 "44"
101 "61"
102 "31"
103 "54"
104 "69"
105 "111"
106 "51"
107 "40"
108 "82"
109 "63"
110 "60"
111 "99"
*States
1 19 "{eps}_1"
2 22 "{1}_2"
3 21 "{eps}_3"
4 10 "{3}_4"
5 10 "{1}_4"
6 10 "{1-2}_4"
